In [1]:
!pip install gym


[notice] A new release of pip available: 22.2.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [50]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 2)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x
      
    def put_data(self, item):
        self.data.append(item)
        
    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -prob * R
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        self.data = []

def main():
    env = gym.make('CartPole-v1')
    pi = Policy()
    score = 0.0
    print_interval = 20
    
    
    for n_epi in range(10000):
        obs = env.reset()
        done = False
        
        while not done: # CartPole-v1 forced to terminates at 500 step.
            obs = torch.tensor(obs[0],dtype=torch.float)
            out = pi(obs)
            m = Categorical(out)
            action = m.sample()
            obs_prime, r, done, info, _ = env.step(action.item())
#             pi.put_data((r,prob[a]))
            pi.put_data((r,torch.log(out[action])))
            obs = obs_prime
            if done:
                break
            score += r
            
        pi.train_net()
        
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()
    
if __name__ == '__main__':
    main()

RuntimeError: both arguments to matmul need to be at least 1D, but they are 0D and 2D